In [4]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint



In [5]:
# Set the path to the dataset directory
data_dir = "G:\\archive\\Dataset"

In [6]:
# Image data augmentation
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

In [7]:
# Load the data
batch_size = 64
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(240, 240),  # EfficientNetB1 input size
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(240, 240),  # EfficientNetB1 input size
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [8]:
# Function to create an EfficientNetB1-based mask classification model
def create_mask_model():
    base_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(240, 240, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(3, activation='softmax')(x)  # 3 classes: Incorrect Mask, With Mask, Without Mask
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [9]:
# Create the mask classification model
mask_model = create_mask_model()

In [10]:
# Compile the model
mask_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
# Train the model
epochs = 10
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

# Use ModelCheckpoint to save the best model weights during training
checkpoint = ModelCheckpoint("efficientnetb1_mask_model.h5", save_best_only=True)

mask_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[checkpoint]
)

Epoch 1/10
37/37 [==============================] - ETA: 0s - loss: 0.2848 - accuracy: 0.8904

C:\Users\shahr\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


37/37 [==============================] - 434s 11s/step - loss: 0.2848 - accuracy: 0.8904 - val_loss: 6.7877 - val_accuracy: 0.6337
Epoch 2/10
37/37 [==============================] - 351s 9s/step - loss: 0.1495 - accuracy: 0.9525 - val_loss: 0.8062 - val_accuracy: 0.9010
Epoch 3/10
37/37 [==============================] - 371s 10s/step - loss: 0.0702 - accuracy: 0.9768 - val_loss: 0.6263 - val_accuracy: 0.8854
Epoch 4/10
37/37 [==============================] - 356s 10s/step - loss: 0.0453 - accuracy: 0.9848 - val_loss: 0.1626 - val_accuracy: 0.9531
Epoch 5/10
37/37 [==============================] - 347s 9s/step - loss: 0.0509 - accuracy: 0.9833 - val_loss: 0.1819 - val_accuracy: 0.9497
Epoch 6/10
37/37 [==============================] - 347s 9s/step - loss: 0.0418 - accuracy: 0.9863 - val_loss: 0.1055 - val_accuracy: 0.9722
Epoch 7/10
37/37 [==============================] - 345s 9s/step - loss: 0.0338 - accuracy: 0.9902 - val_loss: 0.1646 - val_accuracy: 0.9566
Epoch 8/10
37/37 [===

In [12]:
import os

# Get the absolute path of the current directory
current_directory = os.path.abspath(".")

# Check if the model weights file exists in the current directory
model_weights_file = "path_to_save_model.h5"
model_weights_path = os.path.join(current_directory, model_weights_file)

if os.path.exists(model_weights_path):
    print("Model weights are saved at:", model_weights_path)
else:
    print("Model weights file not found.")

Model weights file not found.


In [13]:
import cv2
import numpy as np
import requests
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

In [14]:
# After training the model
mask_model.save("path_to_save_model.h5")

In [15]:
# Later, to load the model
from tensorflow.keras.models import load_model

loaded_model = load_model("path_to_save_model.h5")

In [18]:
# Load the pre-trained mask classification model (MobileNet)
mask_model = load_model("D:\\assignment_3-main\\assignment_3-main\\path_to_save_model.h5")

In [19]:
# Define the labels for mask classification
labels = {0: "Incorrect Mask", 1: "With Mask", 2: "Without Mask"}

In [24]:
# Load the pre-trained face detection model from OpenCV (SSD MobileNet V2)
face_net = cv2.dnn.readNetFromCaffe(
    r"D:\assignment_3-main\assignment_3-main\deploy.prototxt.txt",
    r"D:\assignment_3-main\assignment_3-main\res10_300x300_ssd_iter_140000.caffemodel"
)

In [28]:
# Capture the default webcam (usually webcam index 0)
cap = cv2.VideoCapture(0)

while True:
    # Read the frame from the webcam
    ret, frame = cap.read()

    if not ret:
        break

    # Perform face detection on the frame
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))
    face_net.setInput(blob)
    detections = face_net.forward()

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Check if the face detection confidence is greater than a threshold
        if confidence > 0.5:
            # Compute the (x, y)-coordinates of the bounding box for the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Extract the face region from the frame
            face = frame[startY:endY, startX:endX]

            if face.shape[0] < 10 or face.shape[1] < 10:
                # Ignore very small faces (might be false detections)
                continue

            # Preprocess the face for mask classification
            face = cv2.resize(face, (240, 240))  # Resize to (240, 240)
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # Classify the face using the mask classification model
            prediction = mask_model.predict(face)
            class_id = np.argmax(prediction)
            label = labels[class_id]

            # Draw the label and bounding box on the frame
            label = f"Mask: {label}"
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

    # Display the output frame
    cv2.imshow("Mask Detection", frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 67ms/step
